In [ ]:
# Script for installing latest CUDA for pyTorch
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

In [ ]:
from ultralytics import YOLO
import albumentations
import utils

# Main
import shutil
import os
import glob
import numpy as np
import string
import random
from tqdm import tqdm
tqdm.pandas()
from dataclasses import dataclass

In [ ]:
# Check CUDA
device = utils.get_cuda_device()

In [ ]:
@dataclass
class CFG():
    batch_size: int = 16
    img_size: int = 224
    epochs: int = 500
    seed: int = 69

random.seed(CFG.seed)
os.environ["PYTHONHASHSEED"] = str(CFG.seed)
np.random.seed(CFG.seed)

In [ ]:
# Labels
TRAIN_PATH = "./data/train"
TEST_PATH = "./data/test"
VAL_PATH = "./data/val"
labels = []
letters = list(string.ascii_uppercase)
labels.extend(letters)
labels.extend(["nothing"])
print(labels)

In [ ]:
# Visualize dataset
utils.plot_sample_imgs(labels, TRAIN_PATH)

In [ ]:
# Create val dataset

# Ensure validation directory exists
os.makedirs(VAL_PATH, exist_ok=True)

# Get all class folders from training data
class_folders = [f for f in os.listdir(TRAIN_PATH) if os.path.isdir(os.path.join(TRAIN_PATH, f))]

# Process each class
for class_name in class_folders:
    print(f"Processing class: {class_name}")
    
    # Create corresponding folder in validation directory
    val_class_path = os.path.join(VAL_PATH, class_name)
    os.makedirs(val_class_path, exist_ok=True)
    
    # Get all images for this class
    train_class_path = os.path.join(TRAIN_PATH, class_name)
    image_files = glob.glob(os.path.join(train_class_path, "*.*"))
    
    # Calculate how many images to move (10%)
    num_to_select = max(1, int(len(image_files) * 0.1))
    
    # Randomly select images
    selected_images = random.sample(image_files, num_to_select)
    
    # Copy selected images to validation folder
    for img_path in selected_images:
        img_filename = os.path.basename(img_path)
        dst_path = os.path.join(val_class_path, img_filename)
        shutil.copy2(img_path, dst_path)
    
    print(f"  Moved {num_to_select} images to validation set")

print("Done! Validation set created at", VAL_PATH)

In [ ]:
model = YOLO("yolo11n-cls.pt")

In [ ]:
results = model.train(
    data         = './data',
    epochs       = CFG.epochs,          
    imgsz        = CFG.img_size,
    batch        = CFG.batch_size,
    amp          = True,
    # ────── geometric / multi‐scale ──────
    degrees      = 20,            # ±20° rotation
    translate    = 0.1,           # ±10% shift
    scale        = 0.2,           # up/down to 50% size
    shear        = 5,             # ±5° shear
    perspective  = 0.001,         # small perspective warp
    multi_scale  = True,          # random resize between 0.5–1.5× during training :contentReference[oaicite:0]{index=0}

    # ────── flips ──────
    flipud       = 0.2,           # 20% chance vertical flip
    fliplr       = 0.5,           # 50% chance horizontal flip

    # ────── color jitter ──────
    hsv_h        = 0.015,         # hue shift limit  
    hsv_s        = 0.7,           # saturation shift limit  
    hsv_v        = 0.4,           # value (brightness) shift limit

    # ────── advanced mixing ──────
    mosaic       = 0.0,           # use Mosaic augmentation  
    mixup        = 0.0,           # 50% MixUp :contentReference[oaicite:1]{index=1}

    # ────── classification‐only augmentations ──────
    auto_augment = 'randaugment', # or 'autoaugment' / 'augmix'
    erasing      = 0.0            # random erase 40% of image regions :contentReference[oaicite:2]{index=2}
)


In [ ]:
trained_model = YOLO("runs/classify/train/weights/best.pt")
trained_model.export(format="onnx")